Work to split document in sequence of equal length to be used by the Flert algorithm with context.

In [1]:
from meddocan.language.pipeline import meddocan_pipeline

nlp = meddocan_pipeline()

In [2]:
text = """Datos del paciente.
Nombre:  Ernesto.
Apellidos: Rivera Bueno.
NHC: 368503.
NASS: 26 63514095.
Domicilio:  Calle Miguel Benitez 90.
Localidad/ Provincia: Madrid.
CP: 28016.
Datos asistenciales.
Fecha de nacimiento: 03/03/1946.
País: España.
Edad: 70 años Sexo: H.
Fecha de Ingreso: 12/12/2016.
Médico:  Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Informe clínico del paciente: Paciente de 70 años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
En la exploración física presenta un buen estado general, con abdomen y genitales normales; tacto rectal compatible con adenoma de próstata grado I/IV.
En la analítica de orina destaca la existencia de 4 hematíes/ campo y 0-5 leucocitos/campo; resto de sedimento normal.
Hemograma normal; en la bioquímica destaca una glucemia de 169 mg/dl y triglicéridos de 456 mg/dl; función hepática y renal normal. PSA de 1.16 ng/ml.
Las citologías de orina son repetidamente sospechosas de malignidad.
En la placa simple de abdomen se valoran cambios degenerativos en columna lumbar y calcificaciones vasculares en ambos hipocondrios y en pelvis.
La ecografía urológica pone de manifiesto la existencia de quistes corticales simples en riñón derecho, vejiga sin alteraciones con buena capacidad y próstata con un peso de 30 g.
En la UIV se observa normofuncionalismo renal bilateral, calcificaciones sobre silueta renal derecha y uréteres arrosariados con imágenes de adición en el tercio superior de ambos uréteres, en relación a pseudodiverticulosis ureteral. El cistograma demuestra una vejiga con buena capacidad, pero paredes trabeculadas en relación a vejiga de esfuerzo. La TC abdominal es normal.
La cistoscopia descubre la existencia de pequeñas tumoraciones vesicales, realizándose resección transuretral con el resultado anatomopatológico de carcinoma urotelial superficial de vejiga.
Remitido por: Ignacio Navarro Cuéllar c/ del Abedul 5-7, 2º dcha 28036 Madrid, España E-mail: nnavcu@hotmail.com.
"""

In [3]:
doc = nlp(text)

In [4]:
from typing import Union, Literal, Optional, List
from pathlib import Path
from spacy.tokens import Doc, Token
import codecs
import warnings
from meddocan.language.utils import minibatch

def doc_to_connl03(
    doc: Doc,
    file: Union[str, Path],
    mode: Literal["w", "a"] = "w",
    write_sentences: bool = True,
    window: Optional[int] = None,
    document_separator_token: Optional[str] = None,
) -> None:

    if write_sentences and window is not None:
        warnings.warn(f"write_sentences argument take precedence over window argument.", UserWarning)

    # ----------- Write a Doc to the given file at the CoNNL03 format.
    if isinstance(file, str):
        file = Path(file)

    def get_line(token: Token) -> str:
        if (tag := token.ent_iob_) in ["", "O"]:
            tag = "O"
        else:
            tag = f"{tag}-{token.ent_type_}"

        text = token.text

        if text[0].encode("utf-8") == codecs.BOM_UTF8:

            # Remove the detected BOOM.
            # If not removed, the ﻿ sign sometimes appears in
            # documents in BIO format opened by vscode.

            text = text[1:]

        line = f"{text} {tag}\n"
        return line
    with file.open(mode=mode, encoding="utf-8", newline="\n") as f:
        lines: List[str] = []

        if document_separator_token is not None:
            document_separator_line = f"{document_separator_token} O\n\n"
            f.write(document_separator_line)

        for sent in doc.sents:
            for token in sent:
                if token.is_space:
                    # We can do that because the previous pipeline split
                    # spaCy spaces of the forms "  \n  " in 3 tokens
                    # ["  ", "\n", " "].
                    continue
                lines.append(get_line(token))
            if write_sentences:
                line = "\n"
            else:
                line = "\\n O\n"
            lines.append(line)

        if write_sentences:
            f.writelines(lines)
        elif window:
            for line_batch in minibatch(lines, window):
                f.writelines(line_batch)
                f.write("\n")
        else:
            joined_lines = "".join((*lines, "\n"))
            f.write(joined_lines)   

In [5]:
from meddocan.language.utils import minibatch

In [6]:
import tempfile

with tempfile.NamedTemporaryFile() as f:
    print(f)
    doc_to_connl03(doc, f.name, write_sentences=False, window=50, document_separator_token="-DOCSTART-")
    f.seek(0)
    print(f.read().decode())

-DOCSTART- O

Datos O
del O
paciente O
. O
\n O
Nombre O
: O
Ernesto O
. O
\n O
Apellidos O
: O
Rivera O
Bueno O
. O
\n O
NHC O
: O
368503 O
. O
\n O
NASS O
: O
26 O
63514095 O
. O
\n O
Domicilio O
: O
Calle O
Miguel O
Benitez O
90 O
. O
\n O
Localidad O
/ O
Provincia O
: O
Madrid O
. O
\n O
CP O
: O
28016 O
. O
\n O
Datos O
asistenciales O
. O

\n O
Fecha O
de O
nacimiento O
: O
03 O
/ O
03 O
/ O
1946 O
. O
\n O
País O
: O
España O
. O
\n O
Edad O
: O
70 O
años O
Sexo O
: O
H O
. O
\n O
Fecha O
de O
Ingreso O
: O
12 O
/ O
12 O
/ O
2016 O
. O
\n O
Médico O
: O
Ignacio O
Navarro O
Cuéllar O
NºCol O
: O
28 O
28 O
70973 O
. O
\n O
Informe O

clínico O
del O
paciente O
: O
Paciente O
de O
70 O
años O
de O
edad O
, O
minero O
jubilado O
, O
sin O
alergias O
medicamentosas O
conocidas O
, O
que O
presenta O
como O
antecedentes O
personales O
: O
accidente O
laboral O
antiguo O
con O
fracturas O
vertebrales O
y O
costales O
; O
intervenido O
de O
enfermedad O
de O
Dupuytren O
en O
mano O
dere

In [7]:
import tempfile

with tempfile.NamedTemporaryFile() as f:
    print(f)
    doc._.to_connl03(f.name, write_sentences=False, window=3, document_separator_token="-DOCSTART-")
    f.seek(0)
    print(f.read().decode())

-DOCSTART- O

Datos O
del O
paciente O

. O
\n O
Nombre O

: O
Ernesto O
. O

\n O
Apellidos O
: O

Rivera O
Bueno O
. O

\n O
NHC O
: O

368503 O
. O
\n O

NASS O
: O
26 O

63514095 O
. O
\n O

Domicilio O
: O
Calle O

Miguel O
Benitez O
90 O

. O
\n O
Localidad O

/ O
Provincia O
: O

Madrid O
. O
\n O

CP O
: O
28016 O

. O
\n O
Datos O

asistenciales O
. O
\n O

Fecha O
de O
nacimiento O

: O
03 O
/ O

03 O
/ O
1946 O

. O
\n O
País O

: O
España O
. O

\n O
Edad O
: O

70 O
años O
Sexo O

: O
H O
. O

\n O
Fecha O
de O

Ingreso O
: O
12 O

/ O
12 O
/ O

2016 O
. O
\n O

Médico O
: O
Ignacio O

Navarro O
Cuéllar O
NºCol O

: O
28 O
28 O

70973 O
. O
\n O

Informe O
clínico O
del O

paciente O
: O
Paciente O

de O
70 O
años O

de O
edad O
, O

minero O
jubilado O
, O

sin O
alergias O
medicamentosas O

conocidas O
, O
que O

presenta O
como O
antecedentes O

personales O
: O
accidente O

laboral O
antiguo O
con O

fracturas O
vertebrales O
y O

costales O
; O
intervenido O

de O
enf

Work on speed up the prediction process.

1. Need to convert spaCy docs to Sentences
2. Link this sentences together (left context, right context etc...)

In [44]:
from typing import Iterator
from flair.data import Sentence


def it_sentences(docs: List[Doc]) -> Iterator[Sentence]:
    for doc in docs:
        for ss in doc.sents:
            tokens = [tok.orth_ for tok in ss]
            fs = Sentence(tokens, use_tokenizer=False)
            yield fs
        fs = Sentence(["-DOCSTART-"], use_tokenizer=False)
        yield fs


In [45]:
flat_sentences = it_sentences([doc, doc, doc])
for sent in flat_sentences:
    print(f"{sent.text!r}")

'Datos del paciente . \n'
'Nombre :   Ernesto . \n'
'Apellidos : Rivera Bueno . \n'
'NHC : 368503 . \n'
'NASS : 26 63514095 . \n'
'Domicilio :   Calle Miguel Benitez 90 . \n'
'Localidad / Provincia : Madrid . \n'
'CP : 28016 . \n'
'Datos asistenciales . \n'
'Fecha de nacimiento : 03 / 03 / 1946 . \n'
'País : España . \n'
'Edad : 70 años Sexo : H . \n'
'Fecha de Ingreso : 12 / 12 / 2016 . \n'
'Médico :   Ignacio Navarro Cuéllar NºCol : 28 28 70973 . \n'
'Informe clínico del paciente : Paciente de 70 años de edad , minero jubilado , sin alergias medicamentosas conocidas , que presenta como antecedentes personales : accidente laboral antiguo con fracturas vertebrales y costales ; intervenido de enfermedad de Dupuytren en mano derecha y by - pass iliofemoral izquierdo ; Diabetes Mellitus tipo II , hipercolesterolemia e hiperuricemia ; enolismo activo , fumador de 20 cigarrillos / día . \n'
'Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasió

In [20]:
from flair.datasets.sequence_labeling import ColumnDataset


flat_sentences = it_sentences([doc, doc, doc])

in_memory = True
banned_sentences = None
total_sentence_count = 0
document_separator_token = "-DOCSTART-"

if in_memory:
    sentences: List[Sentence] = []

    # pointer to previous
    previous_sentence = None
    while True:
        # parse next sentence
        try:
            next_sentence = next(flat_sentences)
        except StopIteration:
            break                          

        # quit if last sentence reached
        if len(next_sentence) == 0:
            break

        sentence = next_sentence
        # check if this sentence is a document boundary
        if sentence.to_original_text() == document_separator_token:
            sentence.is_document_boundary = True

        if not sentence:
            continue

        # skip banned sentences
        if banned_sentences is not None and any(
            [d in sentence.to_plain_string() for d in banned_sentences]
        ):
            continue

        # set previous and next sentence for context
        sentence._previous_sentence = previous_sentence
        sentence._next_sentence = None
        if previous_sentence:
            previous_sentence._next_sentence = sentence

        # append parsed sentence to list in memory
        sentences.append(sentence)

        previous_sentence = sentence

    total_sentence_count = len(sentences)



What we can do now is verify that a model made the prediction taking into account the context and the document separator...

In [10]:
sentences[0].next_sentence(), sentences[2].previous_sentence()

(Sentence: "Nombre :   Ernesto . 
 ",
 Sentence: "Nombre :   Ernesto . 
 ")

In [11]:
from flair.models import SequenceTagger

model = SequenceTagger.load("/home/wave/Project/MedDocAn.worktree/master/continuous_split/experiments/grid_search_1/an_wh_rs_False_dpt_0_emb_beto-cased-context_window_60_FT_True_Ly_-1_seed_1_lr_5e-06_it_10_bs_4_opti_AdamW_pjct_emb_False_sdl_LinearSchedulerWithWarmup_use_crf_False_use_rnn_False_wup_0.1/0/final-model.pt")

2022-09-30 11:37:08,840 loading file /home/wave/Project/MedDocAn.worktree/master/continuous_split/experiments/grid_search_1/an_wh_rs_False_dpt_0_emb_beto-cased-context_window_60_FT_True_Ly_-1_seed_1_lr_5e-06_it_10_bs_4_opti_AdamW_pjct_emb_False_sdl_LinearSchedulerWithWarmup_use_crf_False_use_rnn_False_wup_0.1/0/final-model.pt
2022-09-30 11:37:13,253 SequenceTagger predicts: Dictionary with 89 tags: O, S-TERRITORIO, B-TERRITORIO, E-TERRITORIO, I-TERRITORIO, S-FECHAS, B-FECHAS, E-FECHAS, I-FECHAS, S-EDAD_SUJETO_ASISTENCIA, B-EDAD_SUJETO_ASISTENCIA, E-EDAD_SUJETO_ASISTENCIA, I-EDAD_SUJETO_ASISTENCIA, S-NOMBRE_PERSONAL_SANITARIO, B-NOMBRE_PERSONAL_SANITARIO, E-NOMBRE_PERSONAL_SANITARIO, I-NOMBRE_PERSONAL_SANITARIO, S-NOMBRE_SUJETO_ASISTENCIA, B-NOMBRE_SUJETO_ASISTENCIA, E-NOMBRE_SUJETO_ASISTENCIA, I-NOMBRE_SUJETO_ASISTENCIA, S-SEXO_SUJETO_ASISTENCIA, B-SEXO_SUJETO_ASISTENCIA, E-SEXO_SUJETO_ASISTENCIA, I-SEXO_SUJETO_ASISTENCIA, S-CALLE, B-CALLE, E-CALLE, I-CALLE, S-PAIS, B-PAIS, E-PAIS, I-P

In [13]:
%%timeit
model.predict(sentences)

1.99 s ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
model.predict(sentences, mini_batch_size=100)

1.97 s ± 4.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
model.predict(sentences, mini_batch_size=4)

1.85 s ± 7.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
model.predict(sentences, mini_batch_size=1000)

1.99 s ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
model.predict(sentences)

In [47]:
for token in sentences[0]:
    print(token)
print(sentences[2].get_labels())

Token[0]: "Datos"
Token[1]: "del"
Token[2]: "paciente"
Token[3]: "."
Token[4]: "
"
['Span[2:4]: "Rivera Bueno"'/'NOMBRE_SUJETO_ASISTENCIA' (0.4135), 'Span[5:6]: "
"'/'FECHAS' (0.0117)]


In [33]:
nlp_pred = meddocan_pipeline("/home/wave/Project/MedDocAn.worktree/master/continuous_split/experiments/grid_search_1/an_wh_rs_False_dpt_0_emb_beto-cased-context_window_60_FT_True_Ly_-1_seed_1_lr_5e-06_it_10_bs_4_opti_AdamW_pjct_emb_False_sdl_LinearSchedulerWithWarmup_use_crf_False_use_rnn_False_wup_0.1/0/final-model.pt", mini_batch_size=100)

2022-09-30 12:04:23,766 loading file /home/wave/Project/MedDocAn.worktree/master/continuous_split/experiments/grid_search_1/an_wh_rs_False_dpt_0_emb_beto-cased-context_window_60_FT_True_Ly_-1_seed_1_lr_5e-06_it_10_bs_4_opti_AdamW_pjct_emb_False_sdl_LinearSchedulerWithWarmup_use_crf_False_use_rnn_False_wup_0.1/0/final-model.pt
2022-09-30 12:04:25,469 SequenceTagger predicts: Dictionary with 89 tags: O, S-TERRITORIO, B-TERRITORIO, E-TERRITORIO, I-TERRITORIO, S-FECHAS, B-FECHAS, E-FECHAS, I-FECHAS, S-EDAD_SUJETO_ASISTENCIA, B-EDAD_SUJETO_ASISTENCIA, E-EDAD_SUJETO_ASISTENCIA, I-EDAD_SUJETO_ASISTENCIA, S-NOMBRE_PERSONAL_SANITARIO, B-NOMBRE_PERSONAL_SANITARIO, E-NOMBRE_PERSONAL_SANITARIO, I-NOMBRE_PERSONAL_SANITARIO, S-NOMBRE_SUJETO_ASISTENCIA, B-NOMBRE_SUJETO_ASISTENCIA, E-NOMBRE_SUJETO_ASISTENCIA, I-NOMBRE_SUJETO_ASISTENCIA, S-SEXO_SUJETO_ASISTENCIA, B-SEXO_SUJETO_ASISTENCIA, E-SEXO_SUJETO_ASISTENCIA, I-SEXO_SUJETO_ASISTENCIA, S-CALLE, B-CALLE, E-CALLE, I-CALLE, S-PAIS, B-PAIS, E-PAIS, I-P

In [34]:
doc = nlp_pred(text)

In [57]:
%%timeit
doc = nlp_pred(text)

841 ms ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Ernesto NOMBRE_SUJETO_ASISTENCIA
Rivera Bueno NOMBRE_SUJETO_ASISTENCIA
368503 ID_SUJETO_ASISTENCIA
26 ID_SUJETO_ASISTENCIA
63514095 ID_SUJETO_ASISTENCIA
Calle Miguel Benitez 90 CALLE
Madrid TERRITORIO
28016 TERRITORIO
03/03/1946 FECHAS
España PAIS
70 años EDAD_SUJETO_ASISTENCIA
H SEXO_SUJETO_ASISTENCIA
12/12/2016 FECHAS
Ignacio Navarro Cuéllar NOMBRE_PERSONAL_SANITARIO
28 28 70973 ID_TITULACION_PERSONAL_SANITARIO
70 años EDAD_SUJETO_ASISTENCIA
minero SEXO_SUJETO_ASISTENCIA
jubilado SEXO_SUJETO_ASISTENCIA
Ignacio Navarro Cuéllar NOMBRE_PERSONAL_SANITARIO
c/ del Abedul 5-7, 2º dcha CALLE
28036 TERRITORIO
Madrid TERRITORIO
España PAIS
nnavcu@hotmail.com CORREO_ELECTRONICO


This last results show that the prediction with the Flair Sentences that we have just created is wrong...

In [56]:
for sent in sentences:
    span = sent.get_spans("ner")
    print(span)
    


[Span[4:5]: "
" → FECHAS (0.0117)]
[Span[2:3]: " " → FECHAS (0.0117), Span[3:4]: "Ernesto" → NOMBRE_SUJETO_ASISTENCIA (0.3699), Span[5:6]: "
" → FECHAS (0.0117)]
[Span[2:4]: "Rivera Bueno" → NOMBRE_SUJETO_ASISTENCIA (0.4135), Span[5:6]: "
" → FECHAS (0.0117)]
[Span[2:3]: "368503" → ID_SUJETO_ASISTENCIA (0.7425), Span[4:5]: "
" → FECHAS (0.0117)]
[Span[2:3]: "26" → ID_SUJETO_ASISTENCIA (0.3351), Span[3:4]: "63514095" → ID_SUJETO_ASISTENCIA (0.2378), Span[5:6]: "
" → FECHAS (0.0117)]
[Span[2:3]: " " → FECHAS (0.0117), Span[3:7]: "Calle Miguel Benitez 90" → CALLE (0.8192), Span[8:9]: "
" → FECHAS (0.0117)]
[Span[4:5]: "Madrid" → TERRITORIO (0.9209), Span[6:7]: "
" → FECHAS (0.0117)]
[Span[2:3]: "28016" → TERRITORIO (0.9208), Span[4:5]: "
" → FECHAS (0.0117)]
[Span[3:4]: "
" → FECHAS (0.0117)]
[Span[4:9]: "03 / 03 / 1946" → FECHAS (0.9548), Span[10:11]: "
" → FECHAS (0.0117)]
[Span[2:3]: "España" → PAIS (0.8226), Span[4:5]: "
" → FECHAS (0.0117)]
[Span[2:4]: "70 años" → EDAD_SUJETO_ASISTEN